#### <span style="color:GoldenRod "> Import modules </span>

In [1]:
import numpy as np
from sympy.solvers import solve
from sympy import Symbol
from scipy import constants as sc
from matplotlib import pyplot as plt
from scipy.optimize import fsolve

In [2]:
def sciprint(a):
    print(f"{a:0.2g}")

#### Given Information

$$
n = 1.2 \times 10^6 \sqrt{ \frac{Y}{<\sigma v> V_{hs} \tau} }
$$

This formula gives the density in $m^{-3}$.

Where $Y$ is the fusion yield in J. 

$<\sigma v>$ is the fusion reaction rate in m$^3$.s$^{-1}$

$V_{hs}$ is the volume of the hot-spot m$^3$.

$\tau$ is the burn duration.

We give, based on: https://scitechdaily.com/national-ignition-facility-breakthrough-experiment-puts-researchers-at-threshold-of-fusion-ignition/

$Y = 1.3 \text{ MJ}$

$V_{hs}$ : "a hot-spot the diameter of a human hair"

$\tau$: "for 100 trillionths of a second", where we understand a trillion as $10^{12}$

We give, based on: https://www.nature.com/articles/s41586-021-04281-w

$<\sigma v> = 4.2 \times 10^{20} T_i^{3.6}$ with $T_i$ in keV, and $<\sigma v>$ in cm$^3$.s$^{-1}$.

Finally, based on: https://aip.scitation.org/doi/10.1063/5.0003298

It would seem that $T_i = 4.4 \text{ keV}$ is a good bet.

#### Question 1 -  Evaluate the number density $n$ (be careful about units)

In [3]:
Y = 1.3e6
Ti = 4.4 #[keV]
tau = 100*1e-12
R_hair = (75/2)*1e-6
V_hs = (4/3)*sc.pi*R_hair**3
sigma_v = (4.2e-20*Ti**3.6) * 1e-6

In [4]:
n = 1.2e6*np.sqrt(Y/(sigma_v*V_hs*tau))

sciprint(n)

9.9e+31


$n = 9.9 \times 10^{31} \text{ m}^{-3}$

#### Question 2 - In order to find the ionisation fraction, should we use the Thomas-Fermi model, or the Saha ionization equation?

Thomas-Fermi, because we are outside of the density range where Saha's equation is valid.

#### Question 3 - Use both models to estimate the ionisation fraction.

##### a) For the Saha ionisation equation model, use the "simple" equation given in lecture 2 for x (in Saha_first_impressions.ipynb or Saha_first_impressions.html).  We give the ionisation energies of Deuterium (Ui_De = 15.4667) and Tritium (Ui_Te = 13.603). For the sake of simplicity, you can consider the ionisation fraction for a single virtual specie which would have the ionisation energy: Ui = (Ui_De+Ui_Te)/2.

$$ \frac{x^2}{1-x} = 2.4 \cdot 10^{21} \frac{T^{3/2}}{n} e^{-U_i / k_B T}$$

In [6]:
def rhs_Saha_Chen(n,T,Ui):
    """
    Computes the right hand side of Saha's equation for a given temperature T
    
    n = density (nb.m^-3)
    T = temperature (K)
    Ui = Ionisation energy (J)
    """
    rhs = 2.4*1e21*(T**(3/2)/n)*np.exp(-(Ui/(sc.k*T)))
    return rhs

#Assuming x > 0 forces the solver to return only positive solutions
x = Symbol('x', positive=True) 

def solve_Saha(n,T,Ui):
    """
    Computes the ionisation fraction x for a given temperature T
    
    n = density (nb.m^-3)
    T = temperature (K)
    Ui = Ionisation energy (J)
    """
    #Note: sympy.solvers.solve struggles when rhs is large. So let's use two cases:
    if (rhs_Saha_Chen(n,T,Ui) <= 1e2):
        sol = solve((x**2/(1-x)) - rhs_Saha_Chen(n,T,Ui),x) 
    else:
        sol = [1 - 1/rhs_Saha_Chen(n,T,Ui)] #for x->1, x²/(1-x)-> 1/(1-x)
    if len(sol)==0:
        sol = [np.nan]  #If there is no solution, returns a NaN rather than an empty array.
    return sol

In [7]:
Ui_De = 15.4667
Ui_Te = 13.603

Ui_DT = (Ui_De + Ui_Te)/2 * sc.eV

T = Ti*1e3*sc.eV/sc.k

In [8]:
solve_Saha(n, T, Ui_DT)

[0.906989765277499]

$x_{saha} = 0.907$

##### b) For the Thomas-Fermi model, assume A = 2.5 and Z = 1 (equimolar mixture of deuterium and tritium)

In [9]:
def Zav_TF(Z, A, rho, T):
    """
    Finite Temperature Thomas Fermi Charge State using 
    R.M. More (1985), "Pressure Ionization, Resonances, and the
    Continuity of Bound and Free States", Adv. in Atomic 
    Mol. Phys., Vol. 21, p. 332 (Table IV).
    
    Z = atomic number
    rho = mass density (g/cc)
    T = temperature (eV)
    """

    alpha = 14.3139
    beta = 0.6624
    a1 = 0.003323
    a2 = 0.9718
    a3 = 9.26148e-5
    a4 = 3.10165
    b0 = -1.7630
    b1 = 1.43175
    b2 = 0.31546
    c1 = -0.366667
    c2 = 0.983333
    
    rho1 = rho/ A*Z
    T1 = T/Z**(4./3.)
    Tf = T1/(1 + T1)
    Ac = a1*T1**a2 + a3*T1**a4
    B = -np.exp(b0 + b1*Tf + b2*Tf**7)
    C = c1*Tf + c2
    Q1 = Ac*rho1**B
    Q = (rho1**C + Q1**C)**(1/C)
    x = alpha*Q**beta

    return Z*x/(1 + x + np.sqrt(1 + 2.*x))

In [10]:
A = 2.5
Z = 1

n = 9.867843498781554e33
rho = A*sc.m_p*n *1e3*1e-6 #[g/cc]
T = 4400 #Ti*1e3 #[eV]

In [11]:
rho

41262.92843900966

In [12]:
Zav_TF(Z, A, rho, T)

0.9865621266583642

$x_{TF} = 0.992$

##### c) Briefly comment your results

We see that the Thomas-Fermi model ends up giving a stronger ionisation ratio than the Saha equation. This is because it takes the pressure ionisation into account.

#### 4) What would the wavelength $\lambda$ of a laser need to be in order for it to be able to go through the plasma. Comment your result. 

$$
n_C \text{ [cm$^{-3}$]} = 1.1 \times 10^{21} \left( \frac{1}{\lambda \text{ [µm]}} \right)^2
$$

In order for the laser to go through a plasma with density $n_e \sim 9.9 \times 10^{31} \text{ m}^{-3}$, it needs:

$$
\lambda \text{ [µm]} < \sqrt{\frac{1.1 \times 10^{21}}{n_e \text{ [cm$^{-3}$]}}}
$$

In [13]:
n = 1.2e6*np.sqrt(Y/(sigma_v*V_hs*tau))

In [14]:
np.sqrt(1.1e21/(n*1e-6))

0.0033387601094728683

This would be $\lambda = 3.3 \text{ nm}$. To my knowledge, lasers cannot go that low in wavelength. This suggests that such a dense plasma cannot be created simply by hitting a target with a laser and letting it expand. It has to be heavily compressed.

#### Question 5 - Estimate the coupling constant $\Gamma_{ee}$: Is the plasma strongly or weekly coupled?

In [15]:
n = 1.2e6*np.sqrt(Y/(sigma_v*V_hs*tau))

In [16]:
T = Ti*1e3*sc.eV/sc.k
gamma_ee = sc.e**2 * n**(1/3) / (4*sc.pi*sc.epsilon_0 * sc.k*T) 
sciprint(gamma_ee)

0.015


The plasma is rather weakly coupled.

#### Question 6: Estimate the Fermi energy and compare it to $k_B T$. Are the electrons degenerate? (a nuanced answer is allowed)

$$
E_F \text{ [eV]} = 7.9 \left ( \frac{n_e \text{ [cm$^{-3}$]}}{10^{23}} \right )^{2/3}
$$

In [17]:
n = 1.2e6*np.sqrt(Y/(sigma_v*V_hs*tau))

In [18]:
E_F = 7.9*(n*1e-6 / 1e23)**(2/3)
sciprint(E_F)

sciprint(Ti*1e3)

7.8e+02
4.4e+03


$E_F \sim 780$ eV 

$k_B T \sim 4.4$ keV

$k_B T > E_F$ but not so much that the electrons can be treated classicaly.

#### Question 7: Estimate the Fermi pressure and compare it to the thermal pressure 

$$
P_F \text{ [Mbar]} = 0.5 \left ( \frac{n_e \text{ [cm$^{-3}$]}}{10^{23}} \right )^{5/3} 
$$

In [19]:
P_F = 0.5*((n*1e-6)/1e23)**(5/3) * 1e6 #[Mbar = 10^6 Bar]
sciprint(P_F)

P_th = n*sc.k*T / 1e5 #1 bar = 10^5 Pa]
sciprint(P_th)

4.9e+10
7e+11


The thermal pressure ($7 \times 10^{11}$ bars) dominates over the Fermi Pressure, which is itself not small ($5 \times 10^{10}$ bars). 

This makes a very similar point to the comparison of $E_F$ and $k_B T$.

#### Question 8: Place the NIF experiment in the Temperature-Density diagram. Does this match your previous results?

In [20]:
sciprint(np.log10(Ti*1e3))
sciprint(np.log10(T))

3.6
7.7


![title](Temperature-Density_answer.png)

This corresponds to all of our estimations:

- The plasma is pressured ionised (actually if we plotted the rest of the Saha's equation line in the domain where it is not valid, we would see that our NIF experiment is just above it. This means more than half ionised, but not necessarily completely ionised).
- The NIF experiment is quite safely away from E(coulomb)=kT, which suggests a weak coupling
- The plasma is just above the E(fermi)=kT line, which suggests that quantum effects may still play a role but shouldn't dominate. This fits with our estimations.
- We are quite above the 1Gbar line, which again fits our estimations.